In [46]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000 # no of samples to train on

In [2]:
with open('fra.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [3]:
type(lines)

list

In [4]:
# the above str contains \n at the end of line we are splitting the above string on \n it becomes list

In [5]:
lines[0:2]

['Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)',
 'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)']

In [6]:
len(lines[:min(num_samples, len(lines)-1)])

10000

In [7]:
input_texts = []
target_texts = []
# input_characters contain all the input_characters of eng lang
# target_characters contain all the target_characters of fre lang
input_characters = set()
target_characters = set()

# from above we can see that there are 3tabs(\t) we will split at \t
# after splitting first is english language next is french language next is unknown
for line in lines[:min(num_samples, len(lines)-1)]:
    # now line is str type
    # we will split at \t so we get each line as a list of 3 values
    # first one is input string of eng lan
    # second one is french string of fre lan
    # third is unknown
    input_text, target_text, _ = line.split('\t')
    
    # for eg here input_text = 'Go.' and target_text = 'Va !' for first iteration
    
    # for target_text we add \t as the start and \n as end 
    target_text = '\t' + target_text + '\n'
    # now target_text = '\tVa !\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    # this is for adding all characters of eng lang
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    # this is for adding all characters of fre lang
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
    

In [8]:
input_texts[:5]

['Go.', 'Go.', 'Go.', 'Hi.', 'Hi.']

In [9]:
target_texts[:5]

['\tVa !\n', '\tMarche.\n', '\tBouge !\n', '\tSalut !\n', '\tSalut.\n']

In [10]:
max([len(txt) for txt in target_texts])

59

In [48]:
print('Number of input_texts:', len(input_texts))
print('Number of target_texts:', len(target_texts))

Number of input_texts: 10000
Number of target_texts: 10000


In [23]:
decoder_input_data.shape

(10000, 59, 93)

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [ ]:
# len(input_texts) = len(target_texts)
# encoder is taken as english and decoder is taken as french

# encoder english
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

# decoder french
decoder_input_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [30]:
encoder_input_data.shape

(10000, 15, 71)

In [38]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1.
    
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one time stamp
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder target data ahead by one timestamp
            # and will not include the start character
            decoder_target_data[i, t-1, target_token_index[char]] = 1.
    
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [40]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# we discard encoder_outputs and only keep the states info
encoder_states = [state_h, state_c]

In [43]:
# set up the decoder using encoder_states as initial states
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# we setup our decoder to return full output sentences,
# and to return internal states as well, we don't use the
# return states in the training model, but we will use them in inference
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [44]:
# define the model that will turn 
# encoder_input data and decoder_input data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# we are using categorical_corssentropy because all are one_hot representation
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 93)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 335872      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 256),  358400      input_4[0][0]                    
                                                                 lstm[1][1]                   

In [47]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size,
          epochs=epochs, validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 25s 84ms/step - loss: 1.5096 - accuracy: 0.7055 - val_loss: 1.0279 - val_accuracy: 0.7189
Epoch 2/100
125/125 [==============================] - 9s 68ms/step - loss: 0.8663 - accuracy: 0.7638 - val_loss: 0.8176 - val_accuracy: 0.7776
Epoch 3/100
125/125 [==============================] - 7s 59ms/step - loss: 0.6760 - accuracy: 0.8112 - val_loss: 0.6919 - val_accuracy: 0.8016
Epoch 4/100
125/125 [==============================] - 7s 57ms/step - loss: 0.5806 - accuracy: 0.8323 - val_loss: 0.6393 - val_accuracy: 0.8140
Epoch 5/100
125/125 [==============================] - 8s 61ms/step - loss: 0.5327 - accuracy: 0.8447 - val_loss: 0.5902 - val_accuracy: 0.8278
Epoch 6/100
125/125 [==============================] - 7s 59ms/step - loss: 0.5003 - accuracy: 0.8534 - val_loss: 0.5656 - val_accuracy: 0.8343
Epoch 7/100
125/125 [==============================] - 8s 67ms/step - loss: 0.4711 - accuracy: 0.8606 - val_loss: 0.5418 - val_accuracy

125/125 [==============================] - 7s 60ms/step - loss: 0.0870 - accuracy: 0.9728 - val_loss: 0.5907 - val_accuracy: 0.8733
Epoch 58/100
125/125 [==============================] - 9s 68ms/step - loss: 0.0849 - accuracy: 0.9732 - val_loss: 0.5955 - val_accuracy: 0.8732
Epoch 59/100
125/125 [==============================] - 9s 71ms/step - loss: 0.0836 - accuracy: 0.9734 - val_loss: 0.5973 - val_accuracy: 0.8731
Epoch 60/100
125/125 [==============================] - 8s 62ms/step - loss: 0.0821 - accuracy: 0.9744 - val_loss: 0.6019 - val_accuracy: 0.8742
Epoch 61/100
125/125 [==============================] - 8s 65ms/step - loss: 0.0798 - accuracy: 0.9746 - val_loss: 0.6117 - val_accuracy: 0.8720
Epoch 62/100
125/125 [==============================] - 7s 58ms/step - loss: 0.0777 - accuracy: 0.9753 - val_loss: 0.6109 - val_accuracy: 0.8731
Epoch 63/100
125/125 [==============================] - 7s 59ms/step - loss: 0.0771 - accuracy: 0.9754 - val_loss: 0.6211 - val_accuracy: 0.873

In [32]:
len(input_characters)

71

In [35]:
for i in range(5):
    print('ansar')
print(i)

ansar
ansar
ansar
ansar
ansar
4


In [36]:
i

4

In [27]:
names = ['ansar', 'vamsi']
comp = ['facebook', 'google']
zipped = zip(names, comp)
list(zipped)

[('ansar', 'facebook'), ('vamsi', 'google')]